In [1]:
import torch

import os
import cv2
import pandas as pd
import tqdm as tqdm
import numpy as np
import glob as glob
import time
from xml.etree import ElementTree as et
import matplotlib.pyplot as plt
# from config import CLASSES, RESIZE_TO, TRAIN_DIR, VALID_DIR, BATCH_SIZE
from torch.utils.data import Dataset, DataLoader
from utils.model import train,validate,create_model
from utils.custom_utils import (Averager,collate_fn, get_train_transform, get_valid_transform,SaveBestModel)
from utils.load_img import LoadDataset


C:\Users\Lenovo\anaconda3\envs\fasterrcnn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
usage: ipykernel_launcher.py [-h] [-bs BATCHSIZE] [-img IMGSIZE] [-e EPOCH] [--data DATA] [--backbone BACKBONE]
                             [--weights WEIGHTS]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Lenovo\AppData\Roaming\jupyter\runtime\kernel-e9a738e2-13e1-4adb-81f7-31a3a275bda0.json


Just using function


usage: ipykernel_launcher.py [-h] [-bs BATCHSIZE] [-img IMGSIZE] [-e EPOCH] [--data DATA] [--backbone BACKBONE]
                             [--weights WEIGHTS]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Lenovo\AppData\Roaming\jupyter\runtime\kernel-e9a738e2-13e1-4adb-81f7-31a3a275bda0.json


In [2]:
BATCH_SIZE = 4 # increase / decrease according to GPU memeory
RESIZE_TO = (512,512) # resize the image for training and transforms
NUM_EPOCHS = 2 # number of epochs to train for
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

BACKBONE =  "resnet50"
WEIGHTS = False
# training images and XML files directory
TRAIN_DIR = './data/testingcode/train'

# validation images and XML files directory
VALID_DIR = './data/testingcode/valid'
# classes: 0 index is reserved for background
CLASSES = [
    'background', 'oral'
]
NUM_CLASSES = 2

# whether to visualize images after crearing the data loaders
VISUALIZE_TRANSFORMED_IMAGES = False
SAVE_PLOTS_EPOCH = 1 # save loss plots after these many epochs
SAVE_MODEL_EPOCH = 1 # save model after these many epochs




In [3]:

# location to save model and plots
OUT_DIR = './outputs'
if os.path.isdir(OUT_DIR) == False:
    os.mkdir(OUT_DIR)
index = 1
while os.path.isdir(OUT_DIR+f"/output{index}") == True:
    index+=1
OUT_DIR = OUT_DIR+f"/output{index}"
print(OUT_DIR)
os.mkdir(OUT_DIR)


./outputs/output21


In [4]:
dets = {
    "OutPath":f"output{index}",
    "images_path":TRAIN_DIR,
    "labels_path":VALID_DIR,
    "Class Number":NUM_CLASSES,
    "Class":CLASSES,
    "Images Size":RESIZE_TO,
    "Epochs":NUM_EPOCHS,
    "Batch":BATCH_SIZE,
    
}

In [5]:
train_dataset = LoadDataset(TRAIN_DIR, RESIZE_TO[0], RESIZE_TO[1], CLASSES)
valid_dataset = LoadDataset(VALID_DIR, RESIZE_TO[0], RESIZE_TO[1], CLASSES)
# print(train_dataset.all_images[0])
print(train_dataset.dir_path)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(valid_dataset)}\n")
dets['Number of training samples'] = len(train_dataset)
dets['Number of validate samples'] = len(valid_dataset)

./data/testingcode/train
Number of training samples: 10
Number of validation samples: 10



In [6]:
# initialize the Averager class
train_loss_hist = Averager()
val_loss_hist = Averager()
train_itr = 1
val_itr = 1
# train and validation loss lists to store loss values of all...
# ... iterations till ena and plot graphs for all iterations
train_loss_list = []
val_loss_list = []

In [7]:
model = create_model(num_classes=NUM_CLASSES,backbone=BACKBONE,weights=WEIGHTS)
model = model.to(DEVICE)
# get the model parameters
params = [p for p in model.parameters() if p.requires_grad]
# define the optimizer
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)

# name to save the trained model with
MODEL_NAME = 'model'
# whether to show transformed images from data loader or not
if VISUALIZE_TRANSFORMED_IMAGES:
    from utils import show_tranformed_image
    show_tranformed_image(train_loader)
    
# initialize SaveBestModel class
save_best_model = SaveBestModel()

C:\Users\Lenovo\anaconda3\envs\fasterrcnn\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [9]:
# start the training epochs
for epoch in range(NUM_EPOCHS):
    print(f"\nEPOCH {epoch+1} of {NUM_EPOCHS}")
    # reset the training and validation loss histories for the current epoch
    train_loss_hist.reset()
    val_loss_hist.reset()
    # create two subplots, one for each, training and validation
    figure_1, train_ax = plt.subplots()
    figure_2, valid_ax = plt.subplots()
    # start timer and carry out training and validation
    start = time.time()
    train_loss = train(train_loader, model,train_loss_list,train_itr,train_loss_hist,optimizer)
    val_loss = validate(valid_loader, model,val_loss_list,val_itr,val_loss_hist,optimizer)
    print(f"Epoch #{epoch} train loss: {train_loss_hist.value:.3f}")   
    print(f"Epoch #{epoch} validation loss: {val_loss_hist.value:.3f}")   
    end = time.time()
    print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")
    
    save_best_model(
        val_loss_hist.value, epoch, model, optimizer,OUT_DIR
    )

    if (epoch+1) % SAVE_MODEL_EPOCH == 0: # save model after every n epochs
        torch.save(model.state_dict(), f"{OUT_DIR}/model{index}.pth")
        print('SAVING MODEL COMPLETE...\n')

    if (epoch+1) % SAVE_PLOTS_EPOCH == 0: # save loss plots after n epochs
        train_ax.plot(train_loss, color='blue')
        train_ax.set_xlabel('iterations')
        train_ax.set_ylabel('train loss')
        valid_ax.plot(val_loss, color='red')
        valid_ax.set_xlabel('iterations')
        valid_ax.set_ylabel('validation loss')
        figure_1.savefig(f"{OUT_DIR}/train_loss_{index}.png")
        figure_2.savefig(f"{OUT_DIR}/valid_loss_{index}.png")
        print('SAVING PLOTS COMPLETE...')

    if (epoch+1) == NUM_EPOCHS: # save loss plots and model once at the end
        train_ax.plot(train_loss, color='blue')
        train_ax.set_xlabel('iterations')
        train_ax.set_ylabel('train loss')
        valid_ax.plot(val_loss, color='red')
        valid_ax.set_xlabel('iterations')
        valid_ax.set_ylabel('validation loss')
        figure_1.savefig(f"{OUT_DIR}/train_loss_{index}.png")
        figure_2.savefig(f"{OUT_DIR}/valid_loss_{index}.png")
        torch.save(model.state_dict(), f"{OUT_DIR}/model{index}.pth")

    plt.close('all')


EPOCH 1 of 2
Training


Loss: 1.4111: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:29<00:00,  9.74s/it]


Validating


Loss: 1.2818: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:10<00:00,  3.35s/it]


Epoch #0 train loss: 1.425
Epoch #0 validation loss: 1.304
Took 0.655 minutes for epoch 0

Best validation loss: 1.303879936536153

Saving best model for epoch: 1

SAVING MODEL COMPLETE...

SAVING PLOTS COMPLETE...

EPOCH 2 of 2
Training


Loss: 1.1570: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:27<00:00,  9.13s/it]


Validating


Loss: 1.0261: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.84s/it]


Epoch #1 train loss: 1.231
Epoch #1 validation loss: 1.081
Took 0.599 minutes for epoch 1

Best validation loss: 1.0807401339213054

Saving best model for epoch: 2

SAVING MODEL COMPLETE...

SAVING PLOTS COMPLETE...


In [10]:
train_loss_list

[1.431857943534851,
 1.4333252906799316,
 1.411059856414795,
 1.299916386604309,
 1.2359769344329834,
 1.1569973230361938]

In [11]:
train_df = pd.DataFrame([])
train_df["train_loss"] = train_loss_list
train_df["valid_loss"] = val_loss_list

In [64]:
train_df.to_csv(OUT_DIR+"/result.csv",index=False)

In [42]:
f = open(OUT_DIR+"/opt.txt", "w")
for k,v in dic.items():
    f.write(k+" : "+v+"\n")
f.close()

In [46]:
train_loss_hist.current_total

3.629833936691284

In [14]:
#Read yaml file
# import yaml

# with open("data/data.yaml", "r") as stream:
#     try:
#         data = yaml.safe_load(stream)
#     except yaml.YAMLError as exc:
#         print(exc)
# data['train']